In [1]:
%env MKL_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env OMP_NUM_THREADS=1
%env OPENBLAS_NUM_THREADS=1
%env VECLIB_MAXIMUM_THREADS=1

env: MKL_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1
env: OPENBLAS_NUM_THREADS=1
env: VECLIB_MAXIMUM_THREADS=1


In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from imblearn.pipeline import Pipeline

from tqdm import tqdm

In [3]:
# Evaluation functions
from sklearn.metrics import roc_auc_score, make_scorer
from lift.perc_lift_score import perc_lift_score

# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Samplers
from imblearn.over_sampling import SMOTE, ADASYN
from wgan.imblearn import GANbalancer

In [4]:
from sklearn.datasets import make_classification
X,y = make_classification(n_samples=4000, n_features=20, weights=[0.99,0.01], 
                          n_informative=20, n_redundant=0, n_clusters_per_class=5)

# from wgan.data import load_DMC10
# X, _, y, _, idx_cont, idx_cat, cat_dict = load_DMC10("/Users/hauptjoh/Data/DMC10")

In [ ]:
from experiment.experiment_config import experiment_config
scorers, models, samplers = experiment_config(X, idx_cont=None, idx_cat=None)

In [ ]:
idx_cont=None
idx_cat = None

### Samplers
scorers = {'auc':make_scorer(roc_auc_score, needs_proba=True),
          'TDLift':make_scorer(perc_lift_score, needs_proba=True, percentile=0.1)}

### Models
models = []
models.append(('LR', LogisticRegression(solver='liblinear'), {
    "C": [10]
}))
# models.append(('RF', RandomForestClassifier(), {
#     "n_estimators":[100],
#     "max_features":["sqrt"],
#     "min_samples_leaf":[20]
# }))

### Samplers
from imblearn.over_sampling import SMOTE, ADASYN
from wgan.imblearn import GANbalancer

samplers = []

# GAN
if idx_cont is None:
    idx_cont = list(range(X.shape[1]))

categorical = None
if idx_cat is not None:
    categorical = [(i,
                    np.max(X[:,i])+1,
                    int(min(15., np.ceil(np.max((X[:,i])+1)/2)))
                   )
                    for i in idx_cat]

samplers.append(('cGAN', GANbalancer(
        idx_cont=idx_cont, categorical=categorical, batch_size = 128, critic_iterations=1
), {
    'generator_input'  : [50],
    'generator_layers' : [[70]],
    'critic_layers'    : [[40]],
    'n_iter'           : [40000,100000,1000000]
}))

# baseline
samplers.append(('unbalanced', None, {}))

# SMOTE
samplers.append(('SMOTE', SMOTE(), {
    'k_neighbors':[5,10]
}))

# ADASYN
samplers.append(('ADASYN', ADASYN(), {
    'n_neighbors':[5,10]
}))

In [ ]:
seed = 123

score_outer = {}

for sampler_name, sampler, sampler_grid in tqdm(samplers):
    
    sampler_grid = {'sampler__'+key:item for key, item in sampler_grid.items()}
    
    score_inner = {}

    for model_name, model, model_grid in tqdm(models):

        pipeline = Pipeline(memory='./.cachedir', steps=[
            ('scaler', MinMaxScaler()),
            ('sampler', sampler),
            ('classifier', model)
          ])
        model_grid = {'classifier__'+key:item for key, item in model_grid.items()}
        p_grid = {**sampler_grid, **model_grid}
        
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        outer_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
            
        clf = GridSearchCV(pipeline, param_grid= p_grid, cv=inner_cv, scoring=scorers, refit='auc', 
                           return_train_score=True, iid=False, n_jobs=20, pre_dispatch=40, verbose=1)

        score_inner[model_name] = cross_validate(clf, X=X,y=y,cv=outer_cv , scoring=scorers, return_train_score=True,
                                    return_estimator=True, verbose=1)
    score_outer[sampler_name] = score_inner

  0%|          | 0/1 [00:00<?, ?it/s]

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


In [ ]:
scores = pd.DataFrame([{
  'sampler':sampler_name, 'model':model_name, 
    'auc':np.mean(model["test_auc"]),  'auc_sd':np.std(model["test_auc"]),
    'lift0.1':np.mean(model["test_TDLift"]),  'lift0.1_sd':np.std(model["test_TDLift"]),
} for sampler_name, sampler in score_outer.items()
    for model_name, model in sampler.items()]
)

print(scores)

In [ ]:
tuning_results = {sampler_name:
    {model_name:
    # vstack result DataFrame for each outer fold
        pd.concat([ 
            # Inner CV tuning results as DataFrame
            pd.concat([pd.DataFrame(inner_cv.cv_results_['params']).astype(str), 
                       pd.DataFrame({
                           'mean_test_auc':inner_cv.cv_results_['mean_test_auc'],
                           'std_test_auc':inner_cv.cv_results_['std_test_auc'],
                           'mean_test_TDLift':inner_cv.cv_results_['mean_test_TDLift'],
                           'std_test_TDLift':inner_cv.cv_results_['std_test_TDLift']
                       })
                      ], sort=False, ignore_index=False, axis=1)
            for inner_cv in model['estimator']]).groupby(list(model['estimator'][0].cv_results_['params'][0].keys())).mean().reset_index()
            for model_name, model in sampler.items()}
          for sampler_name, sampler in score_outer.items()}


In [ ]:
tuning_results["cGAN"]["LR"].sort_values(["sampler__n_iter"])

In [ ]:
parameter_scores =  pd.DataFrame([{
  'sampler':sampler_name, 'model':model_name,
    'parameter':param_name,
    'parameter_value':str(param_value),
    'auc':cv.cv_results_['mean_test_auc'][i],  'auc_sd':cv.cv_results_["std_test_auc"][i],
    'lift0.1':cv.cv_results_["mean_test_TDLift"][i],  'lift0.1_sd':cv.cv_results_["std_test_TDLift"][i]
}   for sampler_name, sampler in score_outer.items()
    for model_name, model in sampler.items()
    for cv in model['estimator']
    for i, (param_name, param_value) in enumerate(cv.cv_results_['params'][0].items())
]
)

In [ ]:
parameter_scores

In [ ]:
parameter_scores.groupby(['sampler','model','parameter','parameter_value']).mean()